### steps
1. Create an sample SQL database and table for testing cases
2. performing SQL window on database to solve Consecute number problem

In [10]:
import mysql.connector
from mysql.connector import Error

In [16]:

def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    
    mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")

def connect_msql_database(database):
    """
    Function connect to MYSQL database
    """
    conn = None

    try:
        conn = mysql.connector.connect(host='localhost',
                                       database=f'{database}',
                                       user='root',
                                       password='root')
        if conn.is_connected():
            print(f'Connected to MYSQL {database}')
    
    except Error as e:
        print(e)

    finally:
        if conn is not None and conn.is_connected():
            conn.close()
            print("Connection closed")


if __name__ == '__main__':
    create_msql_database("python_mysql")
    connect_msql_database("python_mysql")



Connected to MYSQL python_mysql
Connection closed
